# Preprocesado de los datos 
## Grado en Ingeniería Informática. Universidad de Burgos
**Autor:** Alicia Olivares Gil

In [1]:
%matplotlib inline 
#para dibujar en el propio notebook
import pandas as pd # se importa pandas como pd
import numpy as np  #numpy como np
import pickle as pk
import matplotlib.pyplot as plt #matplotlib.pyplot como plot

# Leer ficheros csv:

In [2]:
import loadData as ld

path =r'../data/'

datos = ld.load(path)
datos = ld.preprocess(datos)

print(len(datos))
datos.head()

/home/aolivaresg/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


8016982


,DateTime,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,target
5495415,2018-08-28 11:50:12,0.0,13.0,10.0,18.0,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,False
5495416,2018-08-28 11:50:13,0.0,13.0,10.0,18.0,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,False
5495417,2018-08-28 11:50:15,1.0,14.0,11.0,19.0,3.0,9.0,1.0,0.0,0.0,0.0,1.0,0.0,False
5495418,2018-08-28 11:50:16,1.0,14.0,11.0,19.0,2.0,10.0,2.0,0.0,0.0,0.0,1.0,0.0,False
5495419,2018-08-28 11:50:17,1.0,14.0,11.0,19.0,3.0,10.0,1.0,0.0,0.0,0.0,1.0,0.0,False


# Añadir target de los ataques conocidos: 

In [3]:
datos = ld.newSeizure(datos, '2018-11-10 3:36:10', '2018-11-10 3:40:37')
datos = ld.newSeizure(datos, '29/01/2019 6:12:04', '29/01/2019 6:15:37')
datos = ld.newSeizure(datos, '2019-01-29 06:12:00', 15*60)
datos = ld.newSeizure(datos, '2019-02-06 21:41:00', '2019-02-06 22:00:00')

# Normalizar datos en bruto y eliminar ruido: 

In [5]:
from transformers import Normalizer, NoiseFilter, PipelineTransformer, VarianceThresholdPD

In [6]:
columnas_presiones = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6']

noiseFilter = NoiseFilter(minimum=5)
normalizer = Normalizer(max_=100)
vt = VarianceThresholdPD(threshold=0.5)
pipelineTransformer = PipelineTransformer(noiseFilter, normalizer, vt)
datosNormalizados = pipelineTransformer.fit_transform(datos[columnas_presiones])
datosNormalizados = pd.concat([datos['DateTime'], datosNormalizados, datos['target']], axis=1)
datosNormalizados.head()

,DateTime,P1,P2,P3,P4,P5,P6,target
5495415,2018-08-28 11:50:12,0.0,10.655738,8.196721,14.754098,0.0,7.377049,False
5495416,2018-08-28 11:50:13,0.0,10.655738,8.196721,14.754098,0.0,7.377049,False
5495417,2018-08-28 11:50:15,0.0,11.475410,9.016393,15.573770,0.0,7.377049,False
5495418,2018-08-28 11:50:16,0.0,11.475410,9.016393,15.573770,0.0,8.196721,False
5495419,2018-08-28 11:50:17,0.0,11.475410,9.016393,15.573770,0.0,8.196721,False


### Segmentar datos en noches: 

In [7]:
noches_segmentadas = ld.segmentNights(datosNormalizados)
print(len(noches_segmentadas))

124


In [8]:
#buscar noches CON ataque
indices=[]
for i in range(len(noches_segmentadas)): 
    for index, row in noches_segmentadas[i].iterrows(): 
        if(row['target'] == True): 
            indices.append(i)
            break; 

In [9]:
indices

[74, 121]

### Guardar resultado: 

In [8]:
with open('datos_normalizados.pkl','wb') as f:
    pk.dump(noches_segmentadas,f)

In [ ]:
for i in range(len(indices)): 
    nombre = 'diacrisis'+str(i+1)+'.pdd'
    with open(nombre,'wb') as f:
        pk.dump(noches_segmentadas[indices[i]],f)

# Estadísticas móviles: 

In [11]:
from transformers import Normalizer, StatisticsTransformer, ConcatenateTransformer

In [12]:
window = 25

normalizer = Normalizer(max_=100)
mean = PipelineTransformer(StatisticsTransformer(mode='mean', window=window), normalizer)
std = PipelineTransformer(StatisticsTransformer(mode='std', window=window), normalizer)
rang = PipelineTransformer(StatisticsTransformer(mode='range', window=window), normalizer)
concat = ConcatenateTransformer(mean,std,rang)

In [18]:
noches_segmentadas_estadisticas = []
for noche in noches_segmentadas: 
    noche_estadisticas = concat.fit_transform(noche.drop(['DateTime','target'], axis=1))
    noche_estadisticas = pd.concat([noche['DateTime'],noche_estadisticas,noche['target']], axis=1)
    noche_estadisticas = noche_estadisticas.dropna()
    noches_segmentadas_estadisticas.append(noche_estadisticas)

In [20]:
len(noches_segmentadas_estadisticas)

124

### Guardar resultado: 

In [21]:
with open('datos_estadisticos.pkl','wb') as f:
    pk.dump(noches_segmentadas_estadisticas,f)